#### Object detection using Transfer Learning (Caltech101)
1. Load in a pre-trained CNN model trained on a large dataset
2. Freeze parameters (weights) in model’s lower convolutional layers
3. Add custom classifier with several layers of trainable parameters to model
4. Train classifier layers on training data available for task
5. Fine-tune hyper parameters and unfreeze more layers as needed

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image_dataset_from_directory
from keras.layers import Dense, Flatten, MaxPooling2D
from keras.models import Sequential

In [2]:
training_data = image_dataset_from_directory(
	directory="../datasets/Object Detection(Ass6)/caltech-101-img",
	shuffle=True,
	validation_split=0.3,
	seed=42,
	subset='training',
	image_size=(224, 224),
	label_mode='categorical'
)

validation_data = image_dataset_from_directory(
	directory="../datasets/Object Detection(Ass6)/caltech-101-img",
	shuffle=True,
	validation_split=0.3,
	seed=42,
	subset='validation',
	image_size=(224, 224),
	label_mode='categorical'
)

Found 9144 files belonging to 102 classes.
Using 6401 files for training.
Found 9144 files belonging to 102 classes.
Using 2743 files for validation.


In [3]:
def data_augmentation(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    image = tf.image.random_brightness(image, max_delta=0.2)
    return image, label

training_data = training_data.map(data_augmentation)

In [4]:
model_path = "../datasets/Object Detection(Ass6)/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

base_model = VGG16(input_shape=(224, 224, 3),
                   include_top = False, 
                   weights = model_path)

In [ ]:
for layer in base_model.layers:
   layer.trainable = False

In [10]:
model = Sequential([
    base_model,
    # MaxPooling2D(),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(102, activation='softmax')
])

In [11]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 102)            │         6,630 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,327,014 (62.28 MB)

 Trainable params: 1,612,326 (6.15 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [12]:
# adam = Adam(0.001)
model.compile(optimizer="adam", loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
H = model.fit(training_data, validation_data=validation_data, batch_size=64, epochs = 10)

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam(learning_rate=1e-5))

In [ ]:
H_fine_tuning = model.fit(training_data, epochs=5, steps_per_epoch=5, validation_data=validation_data)

In [ ]:
sns.lineplot(H.history)

In [ ]:
predicted_value = model.predict(validation_data)

In [ ]:
labels = list(training_data.class_indices.keys())

In [ ]:
n = 887

plt.imshow(training_data[n])
print("Preditcted: ",labels[np.argmax(predicted_value[n])])
print("Actual: ", labels[np.argmax(validation_data[n])])